# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [39]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [40]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [41]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [42]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [43]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [44]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [45]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [46]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [47]:
from typing import List

def multi_user_completion(
    system_prompt: str,
    user_prompts: List[str],
    model: str = MODEL,
    **kwargs
) -> str:
    messages = [{"role": "system", "content": system_prompt}]
    for p in user_prompts:
        messages.append({"role": "user", "content": p})
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        **kwargs
    )
    return response.choices[0].message.content


In [48]:

def get_links(url, extra_prompts: List[str] = None):
    website = Website(url)
    # İlk prompt her zaman link seçim sistemi
    prompts = [get_links_user_prompt(website)]
    # İsteğe bağlı ek promptlar
    if extra_prompts:
        prompts.extend(extra_prompts)

    content = multi_user_completion(
        system_prompt=link_system_prompt,
        user_prompts=prompts,
        model=MODEL,
        response_format={"type": "json_object"}
    )
    return json.loads(content)

In [49]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/blog/reachy-mini',
 '/spaces',
 '/models',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/kyutai/tts-1.6b-en_fr',
 '/apple/DiffuCoder-7B-cpGRPO',
 '/HuggingFaceTB/SmolLM3-3B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/kontext-community/kontext-relight',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/ilcve21/Sparc3D',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/hackaprompt/Pliny_HackAPrompt_Dataset',
 '/datasets/marcelbinz/Psych-101',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/micr

In [50]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'company blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'company discussion forum',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [51]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [52]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collabor

In [53]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Ayrıca bu yazılar türkce olsun"


In [54]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [55]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n1 day ago\n•\n15.3k\n•\n473\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n12 days ago\n•\n188k\n•\n1.48k\nkyutai/tts-1.6b-en_fr\nUpdated\n1 day ago\n•\n15.9k\n•\n261\napple/DiffuCoder-7B-cpGRPO\nUpdated\n5 days 

In [56]:
def create_brochure(company_name: str, url: str):
    # 1. Brochure içeriği için öncelikle sayfa detaylarını al
    details = get_all_details(url)
    
    # 2. İki aşamalı user prompt listesi hazırla
    brochure_prompts = [
        details,
        "Bu bilgileri kullanarak kısa, esprili bir broşür oluştur ve Markdown formatında döndür. Ayrıca metni Türkçe yaz."
    ]
    
    # 3. multi_user_completion ile çağır
    brochure_md = multi_user_completion(
        system_prompt=system_prompt,
        user_prompts=brochure_prompts,
        model=MODEL
    )
    
    # 4. Çıktıyı göster
    display(Markdown(brochure_md))


In [57]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


# Hugging Face Broşürü 🤗

## Yapay Zeka'nın Yüzü: Hugging Face

Herkese merhaba! Burası, yapay zeka (YZ) dünyasında ***Hugging Face*** yani, geleceği inşa eden öpücük yüzlü topluluğumuz! 😄

### Neden Hugging Face?

- **Modeller?** 1 milyondan fazla model arasında kaybolmak için birebir! (Ya da belki kaybolmamak için bir haritaya ihtiyacınız var! 🗺️)
- **Veri Setleri?** Veri cenneti! 250.000'den fazla veri setine göz atabilir, hangisinin en çok dikkat çekeceğine karar verebilirsiniz! 📊
- **Topluluk?** En iyi topluluk burada! Birbirinden eğlenceli bireylerle YZ'yi keşfedin, paylaşın, birlikte model üretin! 🤝

### Müşteri Portföyü 🏢

50,000'den fazla organizasyon bizden yararlandı. Amazon, Google ve Microsoft gibi devler yanda, siz de bu kulvara hemen katılabilirsiniz. 🚀

### İş İmkanları 🎉

Yeni bir iş arıyorsanız, Hugging Face olarak her zaman yeni yeteneklere kapılarımız açık! Üstelik öpücüklerimiz de ücretsiz! 😘

- **Pozisyonlar:** Yazılım Mühendisi, Veri Bilimci, YZ Araştırmacısı ve daha fazlası!
- **Kültür:** Rahat bir ortam, bolca kahve ve yapay zeka ile harika projeler! (Ama komik şakalarınızı saklayın; yapay zeka onların da farkında!) ☕ 

### Fiyatlandırma 💰

Ne de olsa teknoloji pahalı olabilir! Ama biz, uygun fiyatlarla geniş seçenekler sunuyoruz:
- **PRO Hesap:** 9$/ay’a başlamanın tam zamanı!  
- **Takım Paketi:** 20$/kullanıcı ayda – katılın, birlikte başaralım! 🙌
- **Girişim Modeli:** Kendi tasarımınızdaki özel çözümler için bizimle iletişime geçin.

### Şimdi Başlayın!

Hugging Face ile hemen şimdi yapay zeka dünyasına adım atın! 🚀

Daha fazla bilgi için [web sitemizi ziyaret edin](https://huggingface.co).

Hugging Face – geleceğe öpücük! 😘✨

--- 

Şakalar bir yana, Hugging Face ile yapay zeka dünyasında yerinizi alın, iş birliklerimizle YZ'nın geleceğini birlikte inşa edelim!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [58]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [59]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face - Geleceği Kucaklayan AI Topluluğu! 🤗

## İHALE YAPILIYOR! 🛑
**Hugging Face**’e tüm yapay zeka severler, meraklılar ve robotlarla selfie çekmeyi hayal edenler davetlidir! Burada, makine öğrenimi topluluğu geleceği inşa ediyor, yani siz de dahil olabilirsiniz! 

### 🌟 Neler Sunuyoruz?
1. **Modeller**: 1 milyon + model ile tanışın. İsterseniz THUDM/GLM-4.1V-9B-Thinking ile düşünce gücünüzü artırabilirsiniz. Yoksa black-forest-labs/FLUX.1 ile biraz eğlence mi arıyorsunuz?
   
2. **Veri Setleri**: Tam 250,000+ veri seti ile araştırmalarınızı süpercharge edin. Bu veriler Google'dan, Amazon'dan hatta Microsoft'tan geliyor. Herkes burada!

3. **Uygulamalar**: 400,000+ uygulama ile dolu Hugging Face 'evinde' artık harika AI uygulamalarını birkaç tıklama ile oluşturabilirsiniz! 

4. **Topluluk**: **Hugging Face**'de herkes elini taşın altına koyuyor. Gelin birlikte şarkı söyleyelim! (Şarkı söylemenizde yardımcı olacak bir AI henüz yok ama bir gün olacak).

### 💼 Kariyer Olanakları! 
Hugging Face’de çalışmak, bir araç kutusunun en uç noktasında olmak gibidir – her şey elinizin altında! Eğer:
- **Makine öğrenimi** konusunda tutkuluysanız,
- **Sürekli öğrenmeye** açık birisiyseniz,
- **Yapay zeka** ile dalga geçebilecek kadar cesur bir ruhunuz varsa,
Hugging Face sizinle buluşmayı dört gözle bekliyor. 🎉

### 💌 Müşteri Portföyü 
Burası sadece bir topluluk değil, aynı zamanda inanılmaz şirketlerin de adresi! Amazon, Google ve Microsoft gibi devler burayı tercih ediyor. Kısacası, burada bir şirket bekliyorsanız, bunu kopya çekecek kadar iyi yapmanız gerekiyor! 

### 🤔 Neden Hugging Face?
- **Eğlenceli İş Kültürü**: AI ile şarkı söyleyen takımlar, kahve molasında kod yazan mühendisler... Herkesin mutluluğu öncelikli!
- **Gelecek Vaat Eden Projeler**: Yapay zeka uygulamaları oluşturmak için harika fırsatlar sunuyoruz – ve bazen robotlarla dans etme şansı yakalıyorsunuz!
- **Herkes için Açık Kaynak**: Kodun, modelin ve verilerinizi paylaşmaktan çekinmeyin - ne kadar çok, o kadar çok bağış!

### 🎉 Sonuç
Siz de geleceğe bir adım atmak ve AI dünyasına dalış yapmak istiyorsanız, Hugging Face tam size göre. Gelin topluluğumuza katılın ve geleceği kucaklayın! 

👐 **Hugging Face** - AI dünyasında anı yakalayın!

# Bize Katılın
[HuggingFace](https://huggingface.co) sayfasını ziyaret ederek bizimle bağlantıya geçin! 



In [60]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure: Breathe in the AI Hug!

---

## **Hugging Face – The AI Community Building the Future**

Welcome to Hugging Face, where artificial intelligence gets cozy! Our mission? To revolutionize the world of machine learning while giving it the world's biggest virtual bear hug. 🐻❤️

### **Who Are We?**
We’re a community of AI enthusiasts, builders, and models—much like a software utopia where everyone can have a piece of the AI pie! Our offerings include:

- **1 Million+ Models:** Yes, you read that right. It's like the library of Alexandria but for AI models. Need a model? We have one for *that*!
- **250,000+ Datasets:** The data buffet where you can feast like there's no tomorrow. Who knew datasets could be more filling than Thanksgiving dinner?
- **Spaces for Applications:** Think of our spaces as the digital playground for any budding AI developer or cat video creator. So many applications, so little time!

---

### **Join the Hug**
At Hugging Face, we admit it. We’re a little obsessed with community collaboration. Our vibe? Open source with a side of humor and a sprinkle of friendship. 

**What’s in it for you?**
- Build something amazing with our open-source tools!
- Showcase your work and expand your ML portfolio (experience points for the win!).
- Collaborate with top-notch companies like Google, Microsoft, and... *wait for it*... Grammarly (because who doesn’t want an AI buddy that corrects your grammar?)

---

### **Career Opportunities**
Want to be part of the coolest AI community this side of the internet? Join us! We’re on the lookout for:

- **AI Wizards:** Must have a knack for making machines feel emotions (well, philosophical emotions).
- **Data Whisperers:** You talk to data like it’s your best friend—because it is!
- **Space Cadets:** Help us launch the next big thing in AI Spaces! No rocket science required (it's just coding)! 

#### **Perks of Joining Hugging Face:**
- Work with smart folks (and a lot of pets in Zoom calls).
- Free hugs—virtually, that is! 🤗  
- Opportunities to have your memes featured on our internal Slack (no pressure)!

---

### **The Future is Now!**
With over **50,000 organizations** using Hugging Face, we’re making waves in the AI ocean. Are you ready to surf with us?

### **Let’s Connect!**
Check out our website [Hugging Face](https://huggingface.co) and see how you can be part of this amazing journey! 

Remember: In a world full of algorithms, be someone's variable! 

--- 

### **Contact Us**
- **Website:** [huggingface.co](https://huggingface.co)
- **Follow Us on Social Media:** GitHub, Twitter, LinkedIn, and Discord (We promise, our jokes are way better in person!)

---

So what are you waiting for? Join us and let’s make AI fun, friendly, and just a little *fuzzy*! 🐾

*Disclaimer: No actual hugs will be given during online interactions. Human-to-AI hugs are only theoretical… for now.*

In [61]:
#kendim denedim
import os
from IPython.display import Markdown, display, update_display

def custom_brochure(company_name, url, stream=False, save_to_file=None):
    """
    Şirket broşürü oluşturur ve ekranda gösterir ya da dosyaya kaydeder.
    :param company_name: Şirket adı (örn. "HuggingFace")
    :param url: Web sitesi URL'si (örn. "https://huggingface.co")
    :param stream: True ise akış halinde, False ise toplu yanıt
    :param save_to_file: Dosya yolu, broşürü kaydetmek için (örn. "brochure.md")
    """
    try:
        # Kullanıcı mesajını al
        user_prompt = get_brochure_user_prompt(company_name, url)
        
        # OpenAI isteği
        if stream:
            # Akış modunda
            stream_response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            for chunk in stream_response:
                chunk_content = chunk.choices[0].delta.content or ''
                response += chunk_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        else:
            # Toplu yanıt modunda
            response = openai.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            ).choices[0].message.content
            display(Markdown(response))

        # Dosyaya kaydetme (isteğe bağlı)
        if save_to_file:
            with open(save_to_file, 'w', encoding='utf-8') as f:
                f.write(response)
            print(f"Broşür kaydedildi: {save_to_file}")

        return response

    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
        return None

In [62]:
# Toplu yanıt, dosyaya kaydetmeden
custom_brochure("HuggingFace", "https://huggingface.co", stream=False)

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'team page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Geleceği Kucaklayan Yapay Zeka Topluluğu 🎉

## Kimiz Biz?
Hugging Face, yapay zekânın kalbinde yer alan bir platformdur. Makinelerin bir araya gelerek nasıl daha akıllı hale geleceğini keşfettiğimiz bir zemin. Bugün 50,000’den fazla organizasyon bu toplulukta bizimle birlikte. Ne de olsa, daha çok akıllı robot, daha az dümdüz robot, değil mi? 

## Neler Sunuyoruz? 💼
- **Model Kütüphanesi**: 1 milyonun üzerinde model! Evet, doğru duydunuz. Arkadaşlarınız tekrar tekrar “Bunu nasıl yapabilirsin?” diye sorduğunda, sadece “Hugging Face ile!” diye yanıtlayabilirsiniz. 
- **Veri Setleri**: Makine öğrenimi için 250,000’den fazla veri seti. Üstelik birçoğu ücretsiz! “Bedava mı?” diye sorma; sadece al ve bülbül sesi gibi mutlu ol.
- **Uygulama Geliştirme**: Evde otururken bile bir uygulama yaratmanın yolu. Düşünün, bir robot geliştirip ona “yemek yap” dedikten sonra “ama sadece pizza” diye ekleyebilirsin! 

## Kültürel Hava 🌈
- **Açıklık ve İşbirliği**: Biz burada paylaşımcılıkla ilgileniyoruz. Eğer bir şey bulduysanız, bırakın, dünyaya açalım! 
- **Yaratıcılığı Kucaklamak**: Yaratıcı sansür yok! İçinde “Aman Tanrım, bunu nasıl başardım?” bulduğun her şeyi paylaşabilirsin.
- **Esnek Çalışma Ortamı**: Yılın 365 günü spor yapacağınıza söz veriyoruz… Veya sadece uluslararası uzay istasyonunun sınırlarından yemek siparişi vermek isterseniz de, sorun yok!

## Müşterilerimiz 🪐
Hugging Face’in küçük başarı hikayelerini yazmak için Amazon, Google gibi büyük isimlerle birlikte çalışıyoruz. Eğer geleceği merak ediyorsanız, hala çok genç sayılırsınız. 

## Kariyer Fırsatları 🚀
Eğer siz de robotların evriminde yer almak istiyorsanız doğru yerdesiniz! Bizimle birlikte dümende olmak için şu roller açık:
- **Yazılım Geliştirici**: Kod yazarken kahve içmeyi sevenler!
- **Veri Bilimcisi**: "Veri görmek bir sanat" diyorsanız aramıza bekleriz.
- **Topluluk Yöneticisi**: Bizimle gezen, takım olumlu düşünmeyi başaran arkadaşlar.

## İletişime Geçin! 📬
Hugging Face’e katılın ve geleceği birlikte inşa edelim. Unutma, sen ve ben, birlikte daha güçlüyüz! 

---

**Hugging Face: Yapay Zeka ile Gülümse!**

'# Hugging Face: Geleceği Kucaklayan Yapay Zeka Topluluğu 🎉\n\n## Kimiz Biz?\nHugging Face, yapay zekânın kalbinde yer alan bir platformdur. Makinelerin bir araya gelerek nasıl daha akıllı hale geleceğini keşfettiğimiz bir zemin. Bugün 50,000’den fazla organizasyon bu toplulukta bizimle birlikte. Ne de olsa, daha çok akıllı robot, daha az dümdüz robot, değil mi? \n\n## Neler Sunuyoruz? 💼\n- **Model Kütüphanesi**: 1 milyonun üzerinde model! Evet, doğru duydunuz. Arkadaşlarınız tekrar tekrar “Bunu nasıl yapabilirsin?” diye sorduğunda, sadece “Hugging Face ile!” diye yanıtlayabilirsiniz. \n- **Veri Setleri**: Makine öğrenimi için 250,000’den fazla veri seti. Üstelik birçoğu ücretsiz! “Bedava mı?” diye sorma; sadece al ve bülbül sesi gibi mutlu ol.\n- **Uygulama Geliştirme**: Evde otururken bile bir uygulama yaratmanın yolu. Düşünün, bir robot geliştirip ona “yemek yap” dedikten sonra “ama sadece pizza” diye ekleyebilirsin! \n\n## Kültürel Hava 🌈\n- **Açıklık ve İşbirliği**: Biz burada pay

In [63]:
stream_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'events page', 'url': 'https://www.anthropic.com/events'}, {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'}]}


# Anthropic Broşürü

**Hoşgeldiniz!** Anthropic ile AI dünyasına dalış yapmaya hazırlıklı olun! Burada her şey güvenli, eğlenceli ve bir o kadar da zekice! İşte cennetten düşmüş bir tasarım harikası olan şirketimizi tanıttığımız komik ve eğlenceli broşürümüz.

---

### 📜 **Şirket Hakkında**
Anthropic, yapay zekanın (AI) insanlığın uzun vadeli refahına hizmet etmesi için inşa ediliyor. Yani, evet, burada yapay zeka, sadece "merhaba" demekle kalmıyor; aynı zamanda "seni nasıl kurtarabilirim?" diye sormayı da öğreniyor!  🤖💬

### 🧑‍🤝‍🧑 **Kültür**
Anthropic’te herkes birbirini kucaklıyor - ve AI’yi. Misyonumuz? Güvenli, insan odaklı yapay zeka geliştirmek! This is not just a job; it’s a family reunion with cooler gadgets! 🛠️🎉

### 👥 **Müşterilerimiz**
Kendini bir AI geliştirici olarak düşünen herkes, öğretim kurumları ve şirketler için, yaptığımız her işte kendimizi bulabileceğiniz odak noktasıyız! 🤔💡 Kullanıcılarımız Claude ile saatlerce verimli çalışmanın keyfini çıkarıyor!

### 🛠️ **Ürünler**
**Claude** – Hemen hemen her şeyi yapabilen, bilgi dolu ve güvenilir bir bilgi dostu. Claude Opus 4’ten Claude Sonnet 4 ve daha fazlasına kadar, aşık olacağınız bir AI ailesi ile tanışın! ❤️🤖

### 👩‍💻 **Kariyer**
Bizimle çalışmak için sabırsızlanıyor musun? Birçok açık pozisyonumuz var! Eğer AI ile geleceği inşa etmek istiyorsan, doğru yerdesin. Bu işin sadece para kazanmakla kalmayıp içindeki Da Vinci’yi de ortaya çıkarmakla ilgili olduğunu biliyoruz. 🎨🚀

---

### 👔 **Yetenek Avcıları için Minik İpuçları:**
- **Kendinizi Geliştirin:** Anthropic Academy'ye katılmayı unutmayın! (Evet, bu bir okul ama sıkıcı değil, söz!) 🏫😄
- **Ekip Çalışması:** Claude ile birlikte çalışmak, takım arkadaşlarınla halat çekmeye benzer - her birimiz birlikte çalışmazsak düşebiliriz! 🎉💪

### 🎉 **Sonuç**
Anthropic'e katılın ve yapay zeka dünyasında bir yıldız olun! Burada bilgi güce dönüşür, ve biz de bu gücün şovasını yaparız! 

***Herhangi bir sorunuz mu var? Claude ile bir sohbete başlayın... o şaka yapmakta çok iyi!*** 

---

**İletişim:** [Claude.ai](https://www.anthropic.com)  🌐  
**Bizi Takip Edin:** #ClaudeKomedi #YapayZekaKahkahası

In [64]:
stream_brochure("VBAP", "https://www.vbap.com.tr/tr_TR/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.vbap.com.tr/tr_TR/kurumsal'}, {'type': 'company page', 'url': 'https://www.vbap.com.tr/tr_TR'}, {'type': 'products page', 'url': 'https://www.vbap.com.tr/tr_TR/category/urunler'}, {'type': 'careers page', 'url': 'https://www.vbap.com.tr/tr_TR/bize-ulasin'}, {'type': 'academy page', 'url': 'https://www.vbap.com.tr/tr_TR/vbap-akademi'}, {'type': 'brochures page', 'url': 'https://www.vbap.com.tr/tr_TR/brosurler'}]}


# VBAP Broşürü: İş Hayatınızı Eğlenceli Hale Getirin!

## Hoş Geldiniz!

**VBAP** - Biz işimizi dün değil, bugün yapıyoruz! (Gelecek için planlarımız da var ama o konuda çok gizliyiz.) Sizi SAP, ERP çözümleri, yazılım ve danışmanlıkta yeni bir maceraya davet ediyoruz.

### Neden VBAP? 

1. **Büroda Daha Az Suratı Asık İnsan!**  
   Bizim işimiz, iş süreçlerinizi optimize etmek ve şirketinizin her köşesine neşe katmak! SAP Entegre Bütçe Modülü ile kötü sürprizleri ortadan kaldırın. (Yani, ay sonu hesaplarınızı korkunç bir sürpriz için değil, kutlama için düşünmelisiniz!)

2. **Dijital Dönüşüm Yolu**  
   Akıllı Depo Sistemleri ile depolarınızı mükemmelleştirin! Herkesin hayali olan "zamanı durdurabilen" bir depo sistemi, tam karşınızda!

3. **Proje Yönetiminde Uzmanlık**  
   Ekiplerimiz, uluslararası projelerden edindikleri deneyimlerle bilgi dolu ve eğlenceli bir iş yönetimi sunuyor. Projeniz büyük olacak diye korkmayın; biz doğuştan sorun çözücüyüz!

### Müşterilerimiz Kimler?

- **Sektör Pazartesi**: Otomotivden, finans sektörüne kadar geniş bir yelpazede hizmet vermekteyiz. Kısacası, işinizi bizimle daha eğlenceli hale getirebilirsiniz!
- **İş Ortaklarımız**: weHIRE ile iş birliğimiz sayesinde insan kaynaklarınızı dijitalleştiriyoruz. Artık olumsuz işe alım dönemleri geçmişte kaldı! 

### Şirket Kültürümüz

Atölyemiz, sadece iş yapmaktan daha fazlasını amaçlıyor. Hem kahve molası hem de yaratıcılık molaları ile donatılmış, dostluk ve eğlence dolu bir atmosferde çalışıyoruz. İş arkadaşlarınız, sırf "fikir" değil, aynı zamanda "cümle" üretmenizi sağlamak için burada!

### Kariyer Fırsatları

VBAP ailesine katılın! Orada iş yaşamınızla ilgili yeni bir başlangıç yapmanızı sağlamak için tüm gün yeni yazılımlar geliştiriyoruz! Eğer "dünya yeni teknolojiye aç" diyorsanız, yerimiz tam size göre!

---

### İletişim:

Telefon: +90 224 441 46 44  
E-posta: info@vbap.com.tr  

**VBAP ile iş yaşamınızı eğlenceli hale getirin!** Eğlencenin ve başarının birleştiği adres burası! 🌟

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>